In [1]:
import math
import torch
import ttnn
from llama2.model import ModelArgs
from typing import Tuple

# just a super simple forward without host overhead
from ttcode.lightweightmodule import LightweightModule

2024-12-11 17:17:06.072 | DEBUG    | ttnn:<module>:82 - Initial ttnn.CONFIG:
Config{cache_path=/home/bach/.cache/ttnn,model_cache_path=/home/bach/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}
2024-12-11 17:17:06.132 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.pearson_correlation_coefficient be migrated to C++?
2024-12-11 17:17:06.133 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.Conv1d be migrated to C++?
2024-12-11 17:17:06.134 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.conv2d be migrated to C++?
2024-12-11 17:17:06.135 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.unsq

In [2]:
device_id = 0
device = ttnn.open_device(device_id=device_id)

                 Device | INFO     | Opening user mode device driver

2024-12-11 17:17:06.186 | WARNING  | SiliconDriver   - Unknown board type for chip 0. This might happen because chip is running old firmware. Defaulting to DEFAULT
2024-12-11 17:17:06.186 | WARNING  | SiliconDriver   - Unknown board type for chip 0. This might happen because chip is running old firmware. Defaulting to DEFAULT
2024-12-11 17:17:06.186 | INFO     | SiliconDriver   - Detected 1 PCI device : [0]
2024-12-11 17:17:06.187 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.30.0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0


In [3]:
# RMS Norm imported from a common
from ttcode import rmsnorm

In [4]:
# Load checkpoint dict
import os
checkpoint_dict = torch.load("llama2/configs/stories260K.pth")
model_args = checkpoint_dict['model_args']
print(model_args)
state_dict = checkpoint_dict['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

args = ModelArgs()

{'dim': 64, 'n_layers': 5, 'n_heads': 8, 'n_kv_heads': 4, 'vocab_size': 512, 'multiple_of': 4, 'max_seq_len': 512, 'dropout': 0.05}


In [5]:
batch_size = 1
tokens_num = args.max_seq_len

In [6]:
from llama2.model import precompute_freqs_cis
torch_freqs_cos, torch_freqs_sin = precompute_freqs_cis(args.dim // args.n_heads, args.max_seq_len)
# freqs_cos = ttnn.from_torch(torch_freqs_cos, device=device)
#freqs_sin = ttnn.from_torch(torch_freqs_sin, device=device)
freqs_cos = torch_freqs_cos
freqs_sin = torch_freqs_sin

tensor([0.0000, 0.2500, 0.5000, 0.7500])
10000.0


In [7]:
def reshape_for_broadcast(freqs_cis: ttnn.Tensor, x: ttnn.Tensor):
    ndim = len(x.shape)
    assert 0 <= 1 < ndim
    print(f"{freqs_cis.shape}, {x.shape[1]},{x.shape[-1]}")
    assert freqs_cis.shape == (x.shape[1], x.shape[-1]) 
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    print(freqs_cis.shape)
    print(shape)
    return ttnn.reshape(freqs_cis, shape)

In [8]:
def apply_rotary_emb(
    xq: ttnn.Tensor,
    xk: ttnn.Tensor,
    freqs_cos: ttnn.Tensor,
    freqs_sin: ttnn.Tensor
) -> Tuple[ttnn.Tensor, ttnn.Tensor]:
    print("xq:", xq.shape)
    assert tuple(xq.shape)[0] == 1, "Only works with batch 1 :-C"
    xq = ttnn.reshape(xq, (tuple(xq.shape)[:-1] + (-1,2)))
    # Cannot unbind, cannot slice with [:..], must use ttnn.slice
    # xq_r, xq_i = xq.unbind(-1)
    # Squeeze because to_layour only supports 4D max tensor
    xq = ttnn.squeeze(xq, 0)
    xq = ttnn.to_layout(xq, layout = ttnn.ROW_MAJOR_LAYOUT)
    xq = ttnn.unsqueeze(xq, 0)
    xq_r = ttnn.slice(xq, [0,0,0,0,0], list(tuple(xq.shape)[:-1] + (1,)))
    xq_r = ttnn.squeeze(xq_r, -1)
    print("xq_r:", xq_r.shape)
    # ttnn.deallocate(xq_r)
    xq_i = ttnn.slice(xq, [0,0,0,0,1], list(tuple(xq.shape)[:-1] + (2,)))
    xq_i = ttnn.squeeze(xq_i, -1)    
    print("xq_i:", xq_i.shape)

    xk = ttnn.squeeze(xk, 0)
    xk = ttnn.to_layout(xk, layout = ttnn.ROW_MAJOR_LAYOUT)
    xk = ttnn.unsqueeze(xk, 0)
    xk = ttnn.unsqueeze(xk, 0)
    xk_r = ttnn.slice(xk, [0,0,0,0,0],tuple(xk.shape)[:-1] + (1,))
    xk_r = ttnn.squeeze(xk_r, -1)
    # ttnn.deallocate(xk_r)
    xk_i = ttnn.slice(xk, [0,0,0,0,1], tuple(xk.shape)[:-1] + (2,))
    xk_i = ttnn.squeeze(xk_i, -1)  

    freqs_cos = reshape_for_broadcast(freqs_cos, xq_r)
    freqs_sin = reshape_for_broadcast(freqs_sin, xq_r)

    freqs_sin = ttnn.to_layout(freqs_sin, layout = ttnn.TILE_LAYOUT)
    freqs_cos = ttnn.to_layout(freqs_cos, layout = ttnn.TILE_LAYOUT)

    # apply rotation using real numbers
    xq_r = ttnn.to_layout(xq_r, layout = ttnn.TILE_LAYOUT)
    xq_i = ttnn.to_layout(xq_i, layout = ttnn.TILE_LAYOUT)
    xq_out_r = xq_r * freqs_cos - xq_i * freqs_sin
    xq_out_i = xq_r * freqs_sin + xq_i * freqs_cos

    xk_r = ttnn.to_layout(xk_r, layout = ttnn.TILE_LAYOUT)
    xk_i = ttnn.to_layout(xk_i, layout = ttnn.TILE_LAYOUT)
    xk_out_r = xk_r * freqs_cos - xk_i * freqs_sin
    xk_out_i = xk_r * freqs_sin + xk_i * freqs_cos
    
    # there's no ttnn.stack nor ttnn.flatten :-)
    xq_out_r = ttnn.to_layout(xq_out_r, layout = ttnn.ROW_MAJOR_LAYOUT)
    xq_out_i = ttnn.to_layout(xq_out_i, layout = ttnn.ROW_MAJOR_LAYOUT)
    print(xq_out_r.shape, xq_out_i.shape)
    # Create new dimension
    xq_out_r = ttnn.unsqueeze(xq_out_r, -1)
    # Concatenate along the new dimension
    xq_out = torch.concatenate([xq_out_r, xq_out_i], dim=-1)
    # todo: implement flatten
    print(xq_out.shape)

    # xq_out = ttnn.flatten(xq_out, 3)
    return 

In [9]:
from llama2.model import apply_rotary_emb as apply_rotary_emb_torch

def apply_rotary_emb_host(
    xq: torch.Tensor,
    xk: torch.Tensor,
    freqs_cos: torch.Tensor,
    freqs_sin: torch.Tensor
) -> Tuple[torch.Tensor, torch.Tensor]:
    xq = ttnn.to_torch(xq)
    xk = ttnn.to_torch(xk) 
    xq_out, xk_out = apply_rotary_emb_torch(xq, xk, freqs_cos, freqs_sin)

    return (
        ttnn.from_torch(xq_out, device=device, layout=ttnn.TILE_LAYOUT, dtype=ttnn.bfloat16), 
        ttnn.from_torch(xk_out, device=device, layout=ttnn.TILE_LAYOUT, dtype=ttnn.bfloat16)
    )


In [10]:
def repeat_kv(x: ttnn.Tensor, n_rep: int) -> ttnn.Tensor:
    return ttnn.repeat_interleave(x, dim=2, repeats=n_rep)

In [11]:
class Attention(LightweightModule):
    def __init__(self, args: ModelArgs, state_dict: dict, layer_num, device):
        super().__init__()
        self.state_dict = state_dict
        self.max_batch_size = 1
        self.device = device
        # Indicates the number of heads for the Keys and Values
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        # Indicates the number of heads for the Queries
        self.n_q_heads = args.n_heads
        assert args.n_heads % self.n_kv_heads == 0
        # Indicates the dimension of each head, that is, the part of the embedding that each head will be responsible for
        self.head_dim = args.dim // args.n_heads        
        # Indicates how many times the Keys and Values should be repeated        
        self.n_rep = args.n_heads // self.n_kv_heads
        
        prefix = f"layers.{layer_num}.attention."

        self.wq = ttnn.as_tensor(
            torch.transpose(self.state_dict[f"{prefix}wq.weight"], -2, -1,),
            layout=ttnn.TILE_LAYOUT,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat8_b,
            device=self.device
        )

        self.wk = ttnn.as_tensor(
            torch.transpose(self.state_dict[f"{prefix}wk.weight"], -2, -1,),
            layout=ttnn.TILE_LAYOUT,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat8_b,
            device=self.device
        )
        self.wv = ttnn.as_tensor(
            torch.transpose(self.state_dict[f"{prefix}wv.weight"], -2, -1,),
            layout=ttnn.TILE_LAYOUT,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat8_b,
            device=self.device
        )
        self.wo = ttnn.as_tensor(
            torch.transpose(self.state_dict[f"{prefix}wo.weight"], -2, -1,),
            layout=ttnn.TILE_LAYOUT,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat8_b,
            device=self.device
        )

        mask = torch.full((1, 1, args.max_seq_len, args.max_seq_len), float("-inf"))
        self.mask = torch.triu(mask, diagonal=1)
        self.mask = ttnn.from_torch(self.mask, device=device, layout=ttnn.TILE_LAYOUT)


    def forward_opt(self, x: ttnn.Tensor, freqs_cos:torch.Tensor, freqs_sin: torch.Tensor):
        ... 

    def forward(self, x: ttnn.Tensor, freqs_cos:torch.Tensor, freqs_sin: torch.Tensor):
        bsz, seqlen, _ = x.shape
        xq = ttnn.linear(
            x,
            self.wq,
            bias=None,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat8_b,
        )
        xq = ttnn.to_layout(xq, layout=ttnn.ROW_MAJOR_LAYOUT)
        xq = ttnn.reshape(xq, (bsz, seqlen, self.n_q_heads, self.head_dim))
        xq = ttnn.to_layout(xq, layout=ttnn.TILE_LAYOUT)


        xk = ttnn.linear(
            x,
            self.wk,
            bias=None,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat8_b,
        )
        xk = ttnn.to_layout(xk, layout=ttnn.ROW_MAJOR_LAYOUT)
        xk = ttnn.reshape(xk, (bsz, seqlen, self.n_kv_heads, self.head_dim))
        xk = ttnn.to_layout(xk, layout=ttnn.TILE_LAYOUT)


        xv = ttnn.linear(
            x,
            self.wv,
            bias=None,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat8_b,
        )
        xv = ttnn.to_layout(xv, layout=ttnn.ROW_MAJOR_LAYOUT)
        xv = ttnn.reshape(xv, (bsz, seqlen, self.n_kv_heads, self.head_dim))
        xv = ttnn.to_layout(xv, layout=ttnn.TILE_LAYOUT)
        print(xq.get_dtype(), xk.get_dtype(), xv.get_dtype())
        
        # Apply RoPE
        xq, xk = apply_rotary_emb_host(xq, xk, freqs_cos, freqs_sin)

        xk = repeat_kv(xk, self.n_rep)
        xv = repeat_kv(xv, self.n_rep)

        # premute instead of transpose 
        # (B, 1, H_Q, Head_Dim) -> (B, H_Q, 1, Head_Dim)
        xq = ttnn.permute(xq, (0, 2, 1, 3))
        # (B, Seq_Len_KV, H_Q, Head_Dim) -> (B, H_Q, Seq_Len_KV, Head_Dim)
        xk = ttnn.permute(xk, (0, 2, 1, 3))
        xv = ttnn.permute(xv, (0, 2, 1, 3))

        xq = ttnn.to_layout(
            xq, 
            layout=ttnn.TILE_LAYOUT, 
            memory_config=ttnn.DRAM_MEMORY_CONFIG)
        xk = ttnn.to_layout(
            xk, 
            layout=ttnn.TILE_LAYOUT, 
            memory_config=ttnn.DRAM_MEMORY_CONFIG)
        xv = ttnn.to_layout(
            xv, 
            layout=ttnn.TILE_LAYOUT, 
            memory_config=ttnn.DRAM_MEMORY_CONFIG)

        print(xq.shape, xk.shape, xv.shape)
        # use flash attention, shape problem
        # xk = ttnn.permute(xk, (0, 1, 3, 2))
        if False:
            output = ttnn.transformer.scaled_dot_product_attention(
                xq, 
                xk, 
                xv, 
                attn_mask=None, 
                is_causal=True
            )
        
        attention_scores = ttnn.matmul(
            xq,
            ttnn.permute(xk, (0, 1, 3, 2)),
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat16,
            # core_grid=ttnn.CoreGrid(y=batch_size, x=num_cores_x),
        )

        attention_scores = ttnn.div(attention_scores, math.sqrt(self.head_dim))
        attention_scores = attention_scores + self.mask[:, :, :seqlen, :seqlen]

        attention_scores = ttnn.softmax(attention_scores, dim=-1)

        output = ttnn.matmul(
            attention_scores,
            xv,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat16
        )

        output = ttnn.permute(output, (0, 2, 1, 3))
        output = ttnn.to_layout(output, layout=ttnn.ROW_MAJOR_LAYOUT)
        output = ttnn.reshape(output, (bsz, seqlen, -1))
        output = ttnn.to_layout(output, layout=ttnn.TILE_LAYOUT)

        output = ttnn.linear(
            output,
            self.wo,
            bias=None,
            memory_config=ttnn.L1_MEMORY_CONFIG,
            dtype=ttnn.bfloat8_b,
        )
        
        return output

In [12]:
layer_num = 0
attention = Attention(args, state_dict, layer_num, device)
# attention(ttnn.from_torch(torch.rand((1,64)), layout=ttnn.TILE_LAYOUT), 1)

x_torch = torch.rand((batch_size,tokens_num,args.dim)) 

x = ttnn.from_torch(
    x_torch,
    layout=ttnn.TILE_LAYOUT, 
    device=device
)

output = attention.forward(x, freqs_cos, freqs_sin)

                 Always | WARNING  | Specifying tile shape for a row major layout is deprecated, and will be removed soon
                 Always | WARNING  | Specifying tile shape for a row major layout is deprecated, and will be removed soon
                 Always | WARNING  | Specifying tile shape for a row major layout is deprecated, and will be removed soon
                 Always | WARNING  | Specifying tile shape for a row major layout is deprecated, and will be removed soon
                 Always | WARNING  | Specifying tile shape for a row major layout is deprecated, and will be removed soon
DataType.BFLOAT16 DataType.BFLOAT16 DataType.BFLOAT16
xq: torch.Size([1, 512, 8, 8])
                 Always | WARNING  | Specifying tile shape for a row major layout is deprecated, and will be removed soon
                 Always | WARNING  | Specifying tile shape for a row major layout is deprecated, and will be removed soon
                 Always | WARNING  | Specifying tile shape fo